In [109]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime
import matplotlib.dates as mdates
import os

In [110]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set the base path to the desired directory on Google Drive
base_path = '/content/drive/MyDrive/Study_1_Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

In [112]:
def process_data(data, columns_to_remove):

    processed_data = data.drop(columns=columns_to_remove).values
    return processed_data

In [113]:
def construct_3d_array(base_dir, participants, simulations, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye):
    """
    Construct 3D array from CSV files.
    """
    num_rows = 180  # Define number of rows to keep (last 180 rows)
    arrays_3d = []

    for participant in participants:
        participant_id = f"{int(participant):02d}"  # Format participant number to two digits

        valid_simulations = []

        for simulation in simulations:
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')

            # Check if all files exist
            if all(os.path.exists(file) for file in [hr_file_path, gsr_file_path, head_file_path, eye_file_path]):
                valid_simulations.append(simulation)

        num_valid_simulations = len(valid_simulations)
        if num_valid_simulations == 0:
            continue  # Skip this participant if no valid simulations are found

        array_3d = np.zeros((num_valid_simulations, num_rows, 47)) # hr=1, gsr=1, head=15-3, eye=41-8 total columns after removing columns= 48

        for s_idx, simulation in enumerate(valid_simulations):
            # Process hr data
            hr_file_path = os.path.join(base_dir, participant_id, simulation, f'HR{simulation.capitalize()}.csv')
            hr_data = read_csv(hr_file_path)
            processed_hr_data = process_data(hr_data, columns_to_remove_hr)
            processed_hr_data = processed_hr_data[-num_rows:]  # Keep only the last 180 rows

            # Process gsr data
            gsr_file_path = os.path.join(base_dir, participant_id, simulation, f'EDA{simulation.capitalize()}_downsampled.csv')
            gsr_data = read_csv(gsr_file_path)
            processed_gsr_data = process_data(gsr_data, columns_to_remove_gsr)
            processed_gsr_data = processed_gsr_data[-num_rows:]  # Keep only the last 180 rows

            # Process head data
            head_file_path = os.path.join(base_dir, participant_id, simulation, 'head_tracking_downsampled.csv')
            head_data = read_csv(head_file_path)
            processed_head_data = process_data(head_data, columns_to_remove_head)
            processed_head_data = processed_head_data[-num_rows:]  # Keep only the last 180 rows

            # Process eye data
            eye_file_path = os.path.join(base_dir, participant_id, simulation, 'eye_tracking_downsampled.csv')
            eye_data = read_csv(eye_file_path)
            processed_eye_data = process_data(eye_data, columns_to_remove_eye)
            processed_eye_data = processed_eye_data[-num_rows:]  # Keep only the last 180 rows

            # Combine processed data
            combined_data = np.concatenate((processed_hr_data, processed_gsr_data, processed_head_data, processed_eye_data), axis=1)

            array_3d[s_idx, :, :] = combined_data

        arrays_3d.append(array_3d)

    return arrays_3d


In [114]:
sample_size=60
# simulations_train = ['noise','bumps']
# simulations_test=['flat']
# val_indices = [4, 10, 11, 26, 28, 31, 33, 37] # for flat
# train_indices = [0, 1, 2, 3, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 29, 30, 32, 34, 35, 36, 38, 39, 40, 41] # for flat


# simulations_test=['noise']
# simulations_train = ['flat','bumps']
# val_indices = [7, 15, 17, 19, 28, 31, 32, 42, 44, 48] # for noise
# train_indices = [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47] # for noise

simulations_test=['bumps']
simulations_train = ['flat','noise']
val_indices = [1, 12, 16, 18, 22, 26, 28, 37, 41] # for speedbumps
train_indices = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19, 20, 21, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 42, 43, 44] # for speedbumps

In [115]:
participants = [str(i) for i in range(1, 27)]
columns_to_remove_hr = []
columns_to_remove_gsr = []
columns_to_remove_eye = ['#Frame','Time', 'Unnamed: 40','ConvergenceValid','Left_Eye_Closed','Right_Eye_Closed','LocalGazeValid','WorldGazeValid']
columns_to_remove_head = ['#Frame','Time', 'Unnamed: 14']

In [116]:
arrays_train = construct_3d_array(base_path, participants, simulations_train, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)
arrays_test = construct_3d_array(base_path, participants, simulations_test, columns_to_remove_hr, columns_to_remove_gsr, columns_to_remove_head, columns_to_remove_eye)


In [117]:
# Concatenate arrays along the first axis
input_train = np.concatenate(arrays_train, axis=0)
input_test = np.concatenate(arrays_test, axis=0)

# Display the shape of the final concatenated 3D array
print(f"Shape of the final concatenated 3D array: {input_train.shape}")
print(f"Shape of the final concatenated 3D array: {input_test.shape}")

Shape of the final concatenated 3D array: (45, 180, 47)
Shape of the final concatenated 3D array: (23, 180, 47)


In [118]:
def calculate_total_ssq(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    n_columns = [0, 5, 6, 7, 8, 14, 15]
    o_columns = [0, 1, 2, 3, 4, 8, 10]
    d_columns = [4, 7, 9, 10, 11, 12, 13]

    # Calculate sum for each specified set of columns
    n_val = df.iloc[:, n_columns].sum(axis=1)
    o_val = df.iloc[:, o_columns].sum(axis=1)
    d_val = df.iloc[:, d_columns].sum(axis=1)

    return n_val,o_val,d_val

In [119]:
def merge_ssq_column(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          df = pd.read_csv(csv_path)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq_values.append([n_val, o_val, d_val])
          #ssq_values_participant = df.iloc[:, 0:17].values.flatten()   # Assuming SSQ values are in columns 1 to 16
          #total_ssq_values.append(ssq_values_participant)
  ssq_array = np.array(total_ssq_values)
  return ssq_array

def merge_total_ssq(conditions,participants):
  directories = []
  total_ssq_values = []
  for participant in participants:
      participant = f"{int(participant):02d}"
      for condition in conditions:
          directory = os.path.join(base_path, participant, condition)
          directories.append(directory)

  # Loop through each directory
  for directory in directories:
      # Check if the directory exists
      if not os.path.exists(directory):
          continue

      # Get all CSV files in the directory that are named 'ssq.csv'
      csv_files = [file for file in os.listdir(directory) if file == 'ssq.csv']

      # Loop through each CSV file
      for csv_file in csv_files:
          csv_path = os.path.join(directory, csv_file)
          n_val,o_val,d_val = calculate_total_ssq(csv_path)
          total_ssq = (n_val+o_val+d_val) * 3.74
          df = pd.read_csv(csv_path)
          df["total-ssq"] = total_ssq
          #print("csv_path: ",csv_path,"   ",total_ssq)
          total_ssq_values.append(total_ssq)
  # Create a DataFrame from the list of total SSQ values
  df_total_ssq = pd.DataFrame(total_ssq_values, columns=["total-ssq"])
  # Convert the list of total SSQ values to a NumPy array
  total_ssq_array = np.array(total_ssq_values)
  return total_ssq_array



In [120]:
output_train=merge_ssq_column(simulations_train,participants)
output_train = np.squeeze(output_train)
output_test=merge_ssq_column(simulations_test,participants)
output_test = np.squeeze(output_test)
output_train_total_ssq=merge_total_ssq(simulations_train,participants)
output_test_total_ssq=merge_total_ssq(simulations_test,participants)
output_train_total_ssq=output_train_total_ssq.reshape(-1, 1)
output_test_total_ssq=output_test_total_ssq.reshape(-1, 1)
print(output_train.shape,output_test.shape,output_train_total_ssq.shape,output_test_total_ssq.shape)
# print(output_train)
# print(output_train_total_ssq)


(45, 3) (23, 3) (45, 1) (23, 1)


In [121]:
# import numpy as np


# new_data = np.zeros((38, 3))

# # Assigning original serial numbers to the first column
# new_data[:, 0] = np.arange(1, 39)

# # Assigning original values to the second column
# new_data[:, 1] = output_train[:, 0]

# # Sorting the array based on the values (second column)
# sorted_indices = np.argsort(new_data[:, 1])
# sorted_data = new_data[sorted_indices]

# # Assigning new serial numbers to the third column
# current_val = sorted_data[0][1]
# start_idx = 0
# for i, row in enumerate(sorted_data):
#     if row[1] != current_val:
#         sorted_data[start_idx:i, 2] = np.arange(start_idx + 1, i + 1)
#         start_idx = i
#         current_val = row[1]

# # Assigning new serial numbers for the last group
# sorted_data[start_idx:, 2] = np.arange(start_idx + 1, len(sorted_data) + 1)

# #print(sorted_data)


In [122]:
def scale_input_data(input_train, input_test):
    # Get the shape of the input data
    num_samples_train, time_steps_train, num_features = input_train.shape
    num_samples_test, time_steps_test, _ = input_test.shape

    # Reshape the input data into 2D arrays
    flattened_train_data = input_train.reshape(-1, num_features)
    flattened_test_data = input_test.reshape(-1, num_features)

    # Initialize a MinMaxScaler object
    scaler = MinMaxScaler()

    # Fit the scaler on the training data and transform both train and test data
    scaled_train_data = scaler.fit_transform(flattened_train_data)
    scaled_test_data = scaler.transform(flattened_test_data)

    # Reshape the scaled data back to its original shape
    scaled_train_data = scaled_train_data.reshape(num_samples_train, time_steps_train, num_features)
    scaled_test_data = scaled_test_data.reshape(num_samples_test, time_steps_test, num_features)

    return scaled_train_data, scaled_test_data

def scale_target_var(target_data):
    min_val, max_val = np.min(target_data, axis=0), np.max(target_data, axis=0)
    target_data = (target_data-min_val)/(max_val-min_val)

    return target_data, min_val, max_val

In [123]:
input_train, input_test= scale_input_data(input_train[:, (60-sample_size):(180-sample_size), :], input_test[:, (60-sample_size):(180-sample_size), :])
output_train, min_val, max_val = scale_target_var(output_train)

In [124]:
input_val = input_train[val_indices]
input_train = input_train[train_indices]
output_val = output_train[val_indices]
output_train = output_train[train_indices]

In [125]:
print(input_train.shape,input_test.shape)

(36, 120, 47) (23, 120, 47)


In [126]:
from keras.layers import Input, GRU, Dense, Dropout, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping
import sklearn

rmse_n=[]
rmse_o=[]
rmse_d=[]
for iteration in range(5):
  for subclass in range (3):
    def get_gru(input_shape1, input_shape2):
        model = Sequential()
        model.add(GRU(64, input_shape = (input_shape1, input_shape2), return_sequences=False))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        return model


    # Assuming train_input, train_output, test_input, test_output are numpy arrays

    # Reshape train and test inputs to match GRU input shape
    train_input_reshaped = input_train.reshape((input_train.shape[0], input_train.shape[1], input_train.shape[2]))
    test_input_reshaped = input_test.reshape((input_test.shape[0], input_test.shape[1], input_test.shape[2]))
    val_input_reshaped = input_val.reshape((input_val.shape[0], input_val.shape[1], input_val.shape[2]))

    # Get input and output layers for GRU model
    model = get_gru(input_train.shape[1], input_train.shape[2])

    model.compile(loss='mse', optimizer='adam')
    early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5, verbose=1)
    model.fit(train_input_reshaped, output_train[:,subclass], epochs=20, batch_size=32,validation_data=(val_input_reshaped, output_val[:,subclass]), callbacks=[early_stopping])


    pred_test=model.predict(test_input_reshaped)
    pred_test = pred_test*(max_val[subclass]-min_val[subclass]) + min_val[subclass]
    loss = sklearn.metrics.mean_squared_error(pred_test, output_test[:,subclass], squared = False)
    if subclass==0:
      rmse_n.append(loss)
    elif subclass==1:
      rmse_o.append(loss)
    else:
      rmse_d.append(loss)
rmse_combined = np.column_stack((rmse_n, rmse_o, rmse_d))
rmse_n_loss = sum(rmse_n) / len(rmse_n)
rmse_o_loss = sum(rmse_o) / len(rmse_o)
rmse_d_loss = sum(rmse_d) / len(rmse_d)
average_rmse = np.array([rmse_n_loss, rmse_o_loss, rmse_d_loss])

print(rmse_n_loss,rmse_o_loss,rmse_d_loss)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 387ms/step - loss: 0.0640 - val_loss: 0.1149
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0544 - val_loss: 0.1258
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0585 - val_loss: 0.1317
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0436 - val_loss: 0.1107
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0247 - val_loss: 0.0942
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0379 - val_loss: 0.0905
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 0.0348 - val_loss: 0.0974
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0291 - val_loss: 0.1130
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0273 - val_loss: 0.1281
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0402 - val_loss: 0.1208
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0208 - val_loss: 0.0978
Epoch 11: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 395ms/step - loss: 0.0371 - val_loss: 0.1520
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0327 - val_loss: 0.0917
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0287 - val_loss: 0.0908
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0364 - val_loss: 0.0902
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0162 - val_loss: 0.1354
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0359 - val_loss: 0.1191
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0251 - val_loss: 0.0795
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.0139 - val_loss: 0.0744
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.0276 - val_loss: 0.0732
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0234 - val_loss: 0.0717
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0162 - val_loss: 0.0772
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0103 - val_loss: 0.0820
Epoch 13/2

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 422ms/step - loss: 0.0363 - val_loss: 0.1037
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0410 - val_loss: 0.2135
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.0948 - val_loss: 0.1568
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0531 - val_loss: 0.0969
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0304 - val_loss: 0.0964
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0503 - val_loss: 0.0920
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.0421 - val_loss: 0.1050
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.0286 - val_loss: 0.1216
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0389 - val_loss: 0.1084
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.0197 - val_loss: 0.0945
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0267 - val_loss: 0.0909
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.0228 - val_loss: 0.0954
Epoch 13

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 386ms/step - loss: 0.0422 - val_loss: 0.1480
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0416 - val_loss: 0.1054
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.0552 - val_loss: 0.1125
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0664 - val_loss: 0.0948
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 0.0211 - val_loss: 0.1251
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0228 - val_loss: 0.1362
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0365 - val_loss: 0.1069
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0221 - val_loss: 0.0901
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0141 - val_loss: 0.0854
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0121 - val_loss: 0.0848
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.0155 - val_loss: 0.0879
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0137 - val_loss: 0.0918
Epoch 13/

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 422ms/step - loss: 0.0858 - val_loss: 0.1009
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.0817 - val_loss: 0.0889
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0223 - val_loss: 0.1288
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0429 - val_loss: 0.1227
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0269 - val_loss: 0.0969
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0175 - val_loss: 0.0915
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.0166 - val_loss: 0.0939
Epoch 7: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
4.360234804566027 2.727865306487076 2.5334012683936504
